![](images/obspy_logo_full_524x179px.png)

<div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">ObsPy: scaricare i dati di un terremoto</div>

---

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

ObsPy ha clients per accedere direttamente ai dati via...

- FDSN webservices (IRIS, Geofon/GFZ, USGS, NCEDC, SeisComp3 instances, ...)
- ArcLink (EIDA, ...)
- Earthworm
- SeedLink (near-realtime servers)
- NERIES/NERA/seismicportal.eu
- NEIC
- SeisHub (local seismological database)

Questa introduzione mostra come usare il client webservice FDSN. Il webservice FDSN è per adesso il default webservice implementato da molti data centers in tutto il mondo. Clients per altri protocolli lavorano in maniera simile al client FDSN. In questa esercitazione utilizzeremo il FDSN dell'INGV.

#### Dati di forme d'onda

Per questo esempio scegliamo un terremoto avvenuto a Sestola, nell'Appennino Modenese, il 6 Novembre 2017 alle 05:55:53 ora italiana.

In [ ]:
import obspy
from obspy.clients.fdsn import Client

client = Client("INGV")
t = obspy.UTCDateTime("2017-11-06T04:55:53.0")  #Terremoto di Sestola
st = client.get_waveforms("IV", "ZCCA", "", "HHZ",
                          t - 10, t + 60)
print(st)
st.plot()

- di nuovo, i dati delle forma d'onda vengono restitutito come oggetti stream
- per tutti i flussi di lavoro di elaborazione personalizzati, non importa se i dati provengono da un file locale o da un servizio Web


#### Metadati degli eventi 

Il client FDSN può essere usato per richiedere metadati degli eventi: 

In [ ]:
t = obspy.UTCDateTime("2017-11-06T04:55:53.0") #Terremoto di Sestola
catalog = client.get_events(starttime=t - 10 , endtime=t + 10,
                            minmagnitude=2)
print(catalog)
catalog.plot(projection='local');

Le richieste possono avere una vasta gamma di vincoli (vedi [ObsPy Documentation](http://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html)):

- intervallo di tempo
- intervallo geografico (lonlat-box, cerchio con distanza)
- intervallo di profondità
- intervallo di magnitudo, tipo
- agenzia contributiva

#### Metadati delle stationi

Infine, il client FDSN può essere utilizzato per richiedere i metadati della stazione. Le stazioni possono essere consultate utilizzando una vasta gamma di vincoli (vedi [ObsPy documentation](http://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html)):

 * codice network/station
 * intervallo di tempo di funzionamento
 * geografico (lonlat-box, circolare per distanza)

In [ ]:
event = catalog[0]
origin = event.origins[0]


lon = origin.longitude
lat = origin.latitude

# Get currently active stations in 5 km radius around Livermore.
inventory = client.get_stations(longitude=lon, latitude=lat,
                                maxradius=50.0/111, level="channel", 
                                starttime=obspy.UTCDateTime())
print(inventory)
inventory.plot(projection="local", resolution="i");

La parola-chiave **`level=...`** è usata per specificare il livello di dettaglio nell'inventario richiesto

- `"network"`: restituisce solo informazioni sulle reti che corrispondono ai criteri
- `"station"`: restituisce informazioni su tutte le stazioni corrispondenti
- `"channel"`: restituisce le informazioni sui canali disponibili in tutte le reti di stazioni che soddisfano i criteri
- `"response"`: include la risposta dello strumento per tutti i canali corrispondenti (grande dimensione dei dati dei risultati!)

In [ ]:
inventory = client.get_stations(network="IV", station="ERBM",
                                level="station")
print(inventory)

In [ ]:
inventory = client.get_stations(network="IV", station="ERBM",
                                level="channel")
print(inventory)

Per le richieste di forme d'onda che includono la correzione dello strumento, è necessario scaricare anche le informazioni di risposta dello strumento appropriate.

In [ ]:
t = obspy.UTCDateTime("2017-11-06T04:55:53.0")
st = client.get_waveforms("IV", "ZCCA", "", "HHZ", t - 10, t + 60)
inv = client.get_stations(network="IV", station="ZCCA", location="", channel="HHZ",
                          level="response", starttime=t - 10, endtime=t + 10)
st.plot()

st.remove_response(inventory=inv)
st.plot()


### Esercizio: calcolo della magnitudo per il terremoto di Sestola

Applichiamo quanto imparato nell'esempio calcolando la magnitudo locale del terremoto di Sestola

In [ ]:
#tempo origine del terremoto
t = obspy.UTCDateTime("2017-11-06T04:55:53.0")

In [ ]:
#creiamo il catalogo con l'evento
catalog = client.get_events(starttime=t - 10 , endtime=t + 10,
                            minmagnitude=2)

In [ ]:
#creiamo una lista di stazioni vicine
# Get currently active stations in 5 km radius around Livermore.
inventory = client.get_stations(longitude=lon, latitude=lat,
                                maxradius=50.0/111, level="response", 
                                starttime=obspy.UTCDateTime())

In [ ]:
from obspy import Stream
st = Stream()

for network in inventory:
    for station in network:
        try:
            st += client.get_waveforms(network.code, station.code, "*", "HHE",
                                       t - 5 , t + 30)
        except:
            pass

print(st)
st.plot()

In [ ]:
st.detrend('demean')
st.detrend('linear')
st.remove_response(inventory=inventory, water_level=20,output='DISP')
st.plot()

In [ ]:
for t in st:
    print(t.id,t.max())

In [ ]:
# per rimuovere una traccia si può fare nel modo seguente:
#for t in st.select(station='POPM'):
#    st.remove(t)

In [ ]:
for t in st:
    print(t.id,t.max())

In [ ]:
from math import log10
from obspy.geodetics import gps2dist_azimuth

In [ ]:
event = catalog[0]
origin = event.origins[0]
elon = origin.longitude
elat = origin.latitude


In [ ]:
# troviamo le coordinate di una stazione
staz=st[0].id
print(staz)
coords = inventory.get_coordinates(staz)
print(coords)

In [ ]:
# calcoliamo la distanza dall'evento con gps2dist_azimuth(lat1,lon1,lat2,lon2)

dist,az,baz = gps2dist_azimuth(elat,elon,coords['latitude'],coords['longitude'])

In [ ]:
# per esprimere la distanza in km occorre dividere per 1000
dist/1000

# Formula per il calcolo della magnitudo:

# ML = log(AMP) +1.11 * log(DIST) + 0.00189 * DIST + 3.591

## AMP è lo spostamento massimo espresso in mm
## DIST è la distanza stazione - ipocentro espressa in km



In [ ]:
amag = 0.0
for t in st:
    coords = coords = inventory.get_coordinates(t.id)
    DIST,az,baz = gps2dist_azimuth(elat,elon,coords['latitude'],coords['longitude'])
    DIST = DIST/1000
    AMP = abs(t.max()*1000)
    MAG = log10(AMP) +1.11 * log10(DIST) + 0.00189 * DIST + 3.591
    print('ML alla stazione',t.id,' = ','{:4.2f}'.format(MAG))
    amag += MAG   
    

In [ ]:
print ('Media della ML = ','{:4.2f}'.format(amag/len(st)), ' calcolata su ',len(st),' tracce')

#### Esercizio con Client FDSN

Usare il client FDSN per assemblare un piccolo dataset di forme d'onda per un evento.

- cercare un grande terremoto (e.g. per profondità o in una regione a scelta, utilizzare l'opzione **`limit = 5`** per mantenere basso il traffico di rete)

In [ ]:
from obspy.clients.fdsn import Client

client = Client("IRIS")
catalog = client.get_events(minmagnitude=8, limit=5)
print(catalog)
catalog.plot()
event = catalog[0]
print(event)

- cerca le stazioni per vedere le forme d'onda per l'evento. Le stazioni dovrebbero ..
     * essere disponibili al momento dell'evento
     * usare uno stream verticale di 1 Hz ("LHZ", per non sovraccaricare la nostra rete ..)
     * essere a distanza angolare ravvicinata attorno all'evento (ad esempio 90-91 gradi)
     * regolare la ricerca in modo che solo un numero limitato di stazioni (ad esempio 3-6) corrisponda ai criteri di ricerca
     * Una volta che hai trovato un buon set di stazioni, usa `level="response"` poiché avrai bisogno della risposta più tardi.

In [ ]:
origin = event.origins[0]
t = origin.time

inventory = client.get_stations(longitude=origin.longitude, latitude=origin.latitude,
                                minradius=101, maxradius=103,
                                starttime=t, endtime =t+100,
                                channel="LHZ", level="response",
                                matchtimeseries=True)
print(inventory)

- per ognuna di queste stazioni scaricare i dati dell'evento, ad es. da un paio di minuti prima a mezz'ora dopo l'evento
- metti tutti i dati insieme in uno stream (metti la chiamata `get_waveforms()` in un blocco try/except/pass per saltare silenziosamente le stazioni che in realtà non hanno dati disponibili)
- stampa le informazioni di stream, plotta i dati grezzi

In [ ]:
from obspy import Stream
st = Stream()

for network in inventory:
    for station in network:
        try:
            st += client.get_waveforms(network.code, station.code, "*", "LHZ",
                                       t - 5 * 60, t + 30 * 60)
        except:
            pass

print(st)
st.plot()

- correggi la risposta dello strumento per tutte le stazioni e traccia i dati corretti

In [ ]:
st.remove_response(inventory=inventory, water_level=20)
st.plot()

Se hai tempo, assembla e plotta un altro set di dati simile